<a href="https://colab.research.google.com/github/barzansaeedpour/advanced-python-notebooks/blob/main/SOLID_Principles_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLID Principles with Python

When you build a Python project using object-oriented programming (OOP), planning how the different classes and objects will interact to solve your specific problems is an important part of the job. This planning is known as object-oriented design (OOD), and getting it right can be a challenge. If you’re stuck while designing your Python classes, then the SOLID principles can help you out.



SOLID is a set of **five object-oriented design principles** that can help you write more maintainable, flexible, and scalable code based on well-designed, cleanly structured classes. These principles are a fundamental part of object-oriented design best practices.



In this tutorial, you’ll:

- Understand the meaning and purpose of each SOLID principle
- Identify Python code that violates some of the SOLID principles
- Apply the SOLID principles to refactor your Python code and improve its design

Throughout your learning journey, you’ll code practical examples to discover how the SOLID principles can lead to well-organized, flexible, maintainable, and scalable code.

To get the most out of this tutorial, you must have a good understanding of Python object-oriented programming concepts, such as **classes**, **interfaces**, and **inheritance**.

**SOLID** stands for:

- **S**: Single responsibility principle
- **O**: Open/Closed principle
- **L**: Liskov’s substitution principle
- **I**: Interface segregation principle
- **D**: Dependency inversion principle

## Single-Responsibility Principle (SRP)

The single-responsibility principle states that:

**A class should have only one reason to change.**

This means that a class should have only one responsibility, as expressed through its methods. If a class takes care of more than one task, then you should separate those tasks into separate classes.

This principle is closely related to the concept of separation of concerns, which suggests that you should split your programs into different sections. Each section must address a separate concern.

To illustrate the single-responsibility principle and how it can help you improve your object-oriented design, say that you have the following FileManager class:

In [ ]:
# file_manager_srp.py

from pathlib import Path
from zipfile import ZipFile

class FileManager:
    def __init__(self, filename):
        self.path = Path(filename)

    def read(self, encoding="utf-8"):
        return self.path.read_text(encoding)

    def write(self, data, encoding="utf-8"):
        self.path.write_text(data, encoding)

    def compress(self):
        with ZipFile(self.path.with_suffix(".zip"), mode="w") as archive:
            archive.write(self.path)

    def decompress(self):
        with ZipFile(self.path.with_suffix(".zip"), mode="r") as archive:
            archive.extractall()

In this example, your `FileManager` class has two different responsibilities. It uses the `.read()` and `.write()` methods to manage the file. It also deals with ZIP archives by providing the `.compress()` and `.decompress()` methods.

This class violates the single-responsibility principle because it has two reasons for changing its internal implementation. To fix this issue and make your design more robust, you can split the class into two smaller, more focused classes, each with its own specific concern:

In [ ]:
# file_manager_srp.py

from pathlib import Path
from zipfile import ZipFile

class FileManager:
    def __init__(self, filename):
        self.path = Path(filename)

    def read(self, encoding="utf-8"):
        return self.path.read_text(encoding)

    def write(self, data, encoding="utf-8"):
        self.path.write_text(data, encoding)

class ZipFileManager:
    def __init__(self, filename):
        self.path = Path(filename)

    def compress(self):
        with ZipFile(self.path.with_suffix(".zip"), mode="w") as archive:
            archive.write(self.path)

    def decompress(self):
        with ZipFile(self.path.with_suffix(".zip"), mode="r") as archive:
            archive.extractall()

Now you have two smaller classes, each having only a single responsibility. `FileManager` takes care of managing a file, while `ZipFileManager` handles the compression and decompression of a file using the ZIP format. These two classes are smaller, so they’re more manageable. They’re also easier to reason about, test, and debug.

## Open-Closed Principle (OCP)

The open-closed principle (OCP) means that:

**Software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification.**

To understand what the open-closed principle is all about, consider the following Shape class:

In [ ]:
# shapes_ocp.py

from math import pi

class Shape:
    def __init__(self, shape_type, **kwargs):
        self.shape_type = shape_type
        if self.shape_type == "rectangle":
            self.width = kwargs["width"]
            self.height = kwargs["height"]
        elif self.shape_type == "circle":
            self.radius = kwargs["radius"]

    def calculate_area(self):
        if self.shape_type == "rectangle":
            return self.width * self.height
        elif self.shape_type == "circle":
            return pi * self.radius**2

The initializer of Shape takes a shape_type argument that can be either `"rectangle"` or `"circle"`. It also takes a specific set of keyword arguments using the `**kwargs` syntax. If you set the shape type to `"rectangle"`, then you should also pass the `width` and `height` keyword arguments so that you can construct a proper rectangle.

In contrast, if you set the shape type to `"circle"`, then you must also pass a `radius` argument to construct a circle.

Shape also has a `.calculate_area()` method that computes the area of the current shape according to its `.shape_type`:

In [ ]:
rectangle = Shape("rectangle", width=10, height=5)
rectangle.calculate_area()

50

In [ ]:
circle = Shape("circle", radius=5)
circle.calculate_area()


78.53981633974483

The class works. You can create circles and rectangles, compute their area, and so on. However, the class looks pretty bad. Something seems wrong with it at first sight.


Imagine that you need to add a new shape, maybe a square. How would you do that? Well, the option here is to add another elif clause to `.__init__()` and to `.calculate_area()` so that you can address the requirements of a square shape.


Having to make these changes to create new shapes means that your class is open to modification. That violates the open-closed principle. How can you fix your class to make it open to extension but closed to modification? Here’s a possible solution:

In [ ]:
# shapes_ocp.py

from abc import ABC, abstractmethod
from math import pi

class Shape(ABC):
    def __init__(self, shape_type):
        self.shape_type = shape_type

    @abstractmethod
    def calculate_area(self):
        pass

class Circle(Shape):
    def __init__(self, radius):
        super().__init__("circle")
        self.radius = radius

    def calculate_area(self):
        return pi * self.radius**2

class Rectangle(Shape):
    def __init__(self, width, height):
        super().__init__("rectangle")
        self.width = width
        self.height = height

    def calculate_area(self):
        return self.width * self.height

class Square(Shape):
    def __init__(self, side):
        super().__init__("square")
        self.side = side

    def calculate_area(self):
        return self.side**2

In this code, you completely refactored the Shape class, turning it into an abstract base class `(ABC)`. This class provides the required interface (API) for any shape that you’d like to define. That interface consists of a .shape_type attribute and a `.calculate_area()` method that you must override in all the subclasses.

**Note**: The example above and some examples in the next sections use Python’s ABCs to provide what’s called **`interface inheritance`**. In this type of inheritance, subclasses inherit interfaces rather than functionality. In contrast, when classes inherit functionality, then you’re presented with **`implementation inheritance`**.

This update closes the class to modifications. Now you can add new shapes to your class design without the need to modify Shape. In every case, you’ll have to implement the required interface, which also makes your classes `polymorphic`.

## Liskov Substitution Principle (LSP)

The Liskov substitution principle (LSP) was introduced by Barbara Liskov at an OOPSLA conference in 1987. Since then, this principle has been a fundamental part of object-oriented programming. The principle states that:

**Subtypes must be substitutable for their base types.**

For example, if you have a piece of code that works with a Shape class, then you should be able to substitute that class with any of its subclasses, such as Circle or Rectangle, without breaking the code.

In practice, this principle is about making your subclasses behave like their base classes without breaking anyone’s expectations when they call the same methods. To continue with shape-related examples, say you have a Rectangle class like the following:

In [6]:
# shapes_lsp.py

class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def calculate_area(self):
        return self.width * self.height

In Rectangle, you’ve provided the `.calculate_area()` method, which operates with the `.width` and `.height` instance attributes.

Because a square is a special case of a rectangle with equal sides, you think of deriving a Square class from Rectangle in order to reuse the code. Then, you override the setter method for the .width and .height attributes so that when one side changes, the other side also changes:

In [11]:
# shapes_lsp.py

# ...

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

    # We need a setter because, later on, when we try to change either the width or height, we need to change the other one as well.
    # def __setattr__(self, key, value):
    #     super().__setattr__(key, value)
    #     if key in ("width", "height"):
    #         self.__dict__["width"] = value
    #         self.__dict__["height"] = value

In this snippet of code, you’ve defined Square as a subclass of `Rectangle.` As a user might expect, the class `constructor` takes only the side of the square as an argument. Internally, the `.__init__(`) method initializes the parent’s attributes, `.width` and `.height`, with the `side` argument.



You’ve also defined a special method, `.__setattr__()`, to hook into Python’s attribute-setting mechanism and intercept the assignment of a new value to either the `.width` or `.height` attribute. Specifically, when you set one of those attributes, the other attribute is also set to the same value:

In [12]:
square = Square(5)
print(vars(square))
square.width = 7
print(vars(square))

{'width': 5, 'height': 5}
{'width': 7, 'height': 5}


In [13]:
# shapes_lsp.py

# ...

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

    # We need a setter because when later on, we try to change the width or height we need to change both at the same time.
    def __setattr__(self, key, value):
        super().__setattr__(key, value)
        if key in ("width", "height"):
            self.__dict__["width"] = value
            self.__dict__["height"] = value

In [14]:
square = Square(5)
print(vars(square))
square.width = 7
print(vars(square))

{'width': 5, 'height': 5}
{'width': 7, 'height': 7}


Now you’ve ensured that the Square object always remains a valid square, making your life easier for the small price of a bit of wasted memory. Unfortunately, this violates the Liskov substitution principle because you can’t replace instances of `Rectangle` with their `Square` counterparts.

When someone expects a rectangle object in their code, they might assume that it’ll behave like one by exposing two independent `.width` and `.height` attributes. Meanwhile, your Square class breaks that assumption by changing the behavior promised by the object’s interface. That could have surprising and unwanted consequences, which would likely be hard to debug.

While a square is a specific type of rectangle in mathematics, the classes that represent those shapes shouldn’t be in a parent-child relationship if you want them to comply with the Liskov substitution principle. One way to solve this problem is to create a base class for both Rectangle and Square to extend:

In [15]:
# shapes_lsp.py

from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def calculate_area(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def calculate_area(self):
        return self.width * self.height

class Square(Shape):
    def __init__(self, side):
        self.side = side

    def calculate_area(self):
        return self.side ** 2

`Shape` becomes the type that you can substitute through polymorphism with either Rectangle or Square, which are now siblings rather than a parent and a child. Notice that both concrete shape types have distinct sets of attributes, different initializer methods, and could potentially implement even more separate behaviors. The only thing that they have in common is the ability to calculate their area.

With this implementation in place, you can use the `Shape` type interchangeably with its `Square` and `Rectangle` subtypes when you only care about their common behavior:

In [16]:
def get_total_area(shapes):
    return sum(shape.calculate_area() for shape in shapes)

In [17]:
get_total_area([Rectangle(10, 5), Square(5)])

75

Here, you pass a pair consisting of a rectangle and a square into a function that calculates their total area. Because the function only cares about the `.calculate_area()` method, it doesn’t matter that the shapes are different. **This is the essence of the Liskov substitution principle.**

## Resources

- https://realpython.com/solid-principles-python/
- https://medium.com/@m.nusret.ozates/solid-principles-with-python-245e45f9b1f8

Other resources:
- https://www.youtube.com/watch?v=k4efInGWlYI